This notebook runs ECMWF's aifs-single-v1 data-driven model, using ECMWF's [open data](https://www.ecmwf.int/en/forecasts/datasets/open-data) dataset and the [anemoi-inference](https://anemoi-inference.readthedocs.io/en/latest/apis/level1.html) package.

# 1. Install Required Packages and Imports

In [19]:
# Uncomment the lines below to install the required packages
#!pip install -q anemoi-inference[huggingface]==0.4.9 anemoi-models==0.3.1
#!pip install -q earthkit-regrid==0.4.0 ecmwf-opendata 
#!pip install -q flash_attn

In [20]:
import numpy as np
import datetime
from collections import defaultdict
import copy

from typing import Callable
import earthkit.data as ekd
import earthkit.regrid as ekr

from anemoi.inference.runners.simple import SimpleRunner

from ecmwf.opendata import Client as OpendataClient

In [21]:
from torch import Tensor
import torch
from torch.autograd.functional import vjp

# 2. Retrieve Initial Conditions from ECMWF Open Data




### List of parameters to retrieve form ECMWF open data

In [22]:
PARAM_SFC = ["10u", "10v", "2d", "2t", "msl", "skt", "sp", "tcw", "lsm", "z", "slor", "sdor"]
PARAM_SOIL =["vsw","sot"]
PARAM_PL = ["gh", "t", "u", "v", "w", "q"]
LEVELS = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50]
SOIL_LEVELS = [1,2]

### Select a date

In [23]:
DATE = OpendataClient().latest()

In [24]:
print("Initial date is", DATE)

Initial date is 2025-04-11 06:00:00


### Get the data from the ECMWF Open Data API

In [25]:
def get_open_data(param, levelist=[]):
    fields = defaultdict(list)
    # Get the data for the current date and the previous date
    for date in [DATE - datetime.timedelta(hours=6), DATE]:
        data = ekd.from_source("ecmwf-open-data", date=date, param=param, levelist=levelist)
        for f in data:
            # Open data is between -180 and 180, we need to shift it to 0-360
            assert f.to_numpy().shape == (721,1440)
            values = np.roll(f.to_numpy(), -f.shape[1] // 2, axis=1)
            # Interpolate the data to from 0.25 to N320
            values = ekr.interpolate(values, {"grid": (0.25, 0.25)}, {"grid": "N320"})
            # Add the values to the list
            name = f"{f.metadata('param')}_{f.metadata('levelist')}" if levelist else f.metadata("param")
            fields[name].append(values)

    # Create a single matrix for each parameter
    for param, values in fields.items():
        fields[param] = np.stack(values)

    return fields

### Get Input Fields

In [26]:
fields = {}

#### Add the single levels fields

In [27]:
fields.update(get_open_data(param=PARAM_SFC))

In [28]:
soil=get_open_data(param=PARAM_SOIL,levelist=SOIL_LEVELS)

20250411000000-0h-oper-fc.grib2:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

Soil parameters have been renamed since training this model, we need to rename to the original names

In [29]:
mapping = {'sot_1': 'stl1', 'sot_2': 'stl2',
           'vsw_1': 'swvl1','vsw_2': 'swvl2'}
for k,v in soil.items():
    fields[mapping[k]]=v

#### Add the pressure levels fields

In [30]:
fields.update(get_open_data(param=PARAM_PL, levelist=LEVELS))

#### Convert geopotential height into geopotential

In [31]:
# Transform GH to Z
for level in LEVELS:
    gh = fields.pop(f"gh_{level}")
    fields[f"z_{level}"] = gh * 9.80665

### Create Initial State

In [32]:
input_state = dict(date=DATE, fields=fields)

# 3. Load the Model and Run the Forecast

### Download the Model's Checkpoint from Hugging Face & create a Runner

In [33]:
checkpoint = {"huggingface":"ecmwf/aifs-single-1.0"}

To reduce the memory usage of the model certain environment variables can be set, like the number of chunks of the model's mapper.
Please refer to:
- https://anemoi.readthedocs.io/projects/models/en/latest/modules/layers.html#anemoi-inference-num-chunks
- https://pytorch.org/docs/stable/notes/cuda.html#optimizing-memory-usage-with-pytorch-cuda-alloc-conf

for more information. To do so, you can use the code below:
```
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True' 
os.environ['ANEMOI_INFERENCE_NUM_CHUNKS']='16'
```

In [34]:
runner = SimpleRunner(checkpoint, device="cuda")

**Note - changing the device from GPU to CPU**

- Running the transformer model used on the CPU is tricky, it depends on the FlashAttention library which only supports Nvidia and AMD GPUs, and is optimised for performance and memory usage
- In newer versions of anemoi-models, v0.4.2 and above, there is an option to switch off flash attention and uses Pytorchs Scaled Dot Product Attention (SDPA). The code snippet below shows how to overwrite a model from a checkpoint to use SDPA. Unfortunately it's not optimised for memory usage in the same way, leading to much greater memory usage. Please refer to https://github.com/ecmwf/anemoi-inference/issues/119 for more details 

#### Run the forecast

In [35]:
#def model_wrapper(t_input_keys: tuple[Tensor],t_input: tuple[Tensor],extra:list) -> Callable:
def model_wrapper(input_state:dict) -> Callable:    
    
    #
    #t_input:tuple[Tensor]=()
    #t_input_keys=()
    #for k, v in input_state["fields"].items():
    #    t_input += (Tensor(v),)
    #    t_input_keys += (k,)
    
    t_input_keys=()
    for k in input_state["fields"].keys():
        t_input_keys += (k,)

    def model_step(*args) -> tuple[Tensor]:
        print("ARGUMENTS")
        print("  model_wrapper args: number of Tensor in tuple ",len(args))
        #print("  model_wrapper args: norm of 2t ",np.linalg.norm(args[3].detach().numpy())) #index 3 is 2t in t_input
        #Convert tuple[Tensor] in dictionary
        for k,v in zip(t_input_keys, args):
            input_state["fields"][k] = v

        #Call runner.run
        generator_state=runner.run(input_state=input_state, lead_time=12) 
        output_state=next(generator_state)  
        
        print(output_state)
    
        #Convert Dictionary to tuple[Tensor]
        t_output:tuple[Tensor]=()
        t_output_keys=()
        for k,v in output_state["fields"].items():
            t_output += (Tensor(v),)
            t_output_keys += (k,)
        #print("  t_output: number of Tensor in tuple ",len(t_output))
        #print("  t_output_keys ",t_output_keys)
    
        return t_output
    
    return model_step

In [36]:
#Load input as dictionary
print("")
print("LOAD INPUT IN DICTIONARY")
input_state = dict(date=DATE, fields=fields)
print("  input_state: number of fields in fields ",len(input_state["fields"]))
print("  input_state: norm of 2t ",np.linalg.norm(input_state["fields"]["2t"]))

#Not sure why input_state has a different structure when called a second time
#t_input_keys: first call gives ('10u', '10v', '2d', '2t', 'msl',..., 'z_150', 'z_100', 'z_50')
#t_input_keys: second call gives ('10u', '10v', '2d', '2t', 'msl',..., 'z_150', 'z_100', 'z_50', 'cos_latitude', 'cos_longitude', 'sin_latitude', 'sin_longitude', 'cos_julian_day', 'cos_local_time', 'sin_julian_day', 'sin_local_time', 'insolation')


print("  convert dictionary to tuple[Tensor]")
t_input:tuple[Tensor]=()
t_input_keys=()
for k,v in input_state["fields"].items():
    t_input += (Tensor(v),)
    t_input_keys += (k,)
print("  t_input: number of Tensor in tuple ",len(t_input))
print("  t_input: norm of 2t ",np.linalg.norm(t_input[3])) #index 3 is 2t in t_input

#Create perturbation (from the output of the forecast model)
print("")
print("CREATE PERTURBATION")
pert_name="2t"
print("  Integrate the model")
generator_state=runner.run(input_state=input_state, lead_time=12) 
output_state=next(generator_state)
print("  output_state: number of fields in fields ",len(output_state["fields"]))
print("  output_state: norm of 2t ",np.linalg.norm(output_state["fields"]["2t"]))

output_pert=copy.deepcopy(output_state)
for idx,(k,v) in enumerate(output_state["fields"].items()):
    if k==pert_name:
        print("  Perturbing field %s index %s"%(pert_name,idx))
        output_pert["fields"][k]=0.01*output_state["fields"][k]
    else:
        output_pert["fields"][k]=0.0*output_state["fields"][k]
        
print("  output_pert: number of fields in fields ",len(output_pert["fields"]))
print("  output_pert: norm of 2t ",np.linalg.norm(output_pert["fields"]["2t"]))

#Convert dictionary to tuple[Tensor]
print("  convert dictionary to tuple[Tensor]")
t_pert:tuple[Tensor]=()
t_pert_keys=()
for k,v in output_pert["fields"].items():
    t_pert += (Tensor(v),) 
    t_pert_keys += (k,)
print("  t_pert: number of Tensor in tuple ",len(t_pert)) 
print("  t_pert: norm of 2t ",np.linalg.norm(t_pert[81])) #index 81 is 2t in t_output



#Compute derivatives
print("")
print("COMPUTE DERIVATIVES")
with torch.autograd.set_detect_anomaly(False):
            
    t_output, t_dx_output = vjp(
        model_wrapper(input_state),
        t_input,  # "flattened" inputs
        v=t_pert,  # output perturbations
        create_graph=False,
        strict=False,
    )
    
    
    
print(np.linalg.norm(t_output[81]))
print(np.linalg.norm(t_dx_output[3]))  
print(len(t_dx_output))


LOAD INPUT IN DICTIONARY
  input_state: number of fields in fields  94
  input_state: norm of 2t  299821.67010851036
  convert dictionary to tuple[Tensor]
  t_input: number of Tensor in tuple  94
  t_input: norm of 2t  299821.75

CREATE PERTURBATION
  Integrate the model


Fetching 12 files: 100%|██████████| 12/12 [00:00<00:00, 96420.78it/s]


  output_state: number of fields in fields  102
  output_state: norm of 2t  212410.98
  Perturbing field 2t index 81
  output_pert: number of fields in fields  102
  output_pert: norm of 2t  2124.1091
  convert dictionary to tuple[Tensor]
  t_pert: number of Tensor in tuple  102
  t_pert: norm of 2t  2124.1091

COMPUTE DERIVATIVES
ARGUMENTS
  model_wrapper args: number of Tensor in tuple  94


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

**Note** 
Due to the non-determinism of GPUs, users will be unable to exactly reproduce an official AIFS forecast when running AIFS Single themselves.
If you want to enforece determinism at GPU level, you can do so enforcing the following settings:

```
#First in your terminal
export CUBLAS_WORKSPACE_CONFIG=:4096:8

#And then before running inference:
import torch
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True)

```
Using the above approach will significantly increase runtime. Additionally, the input conditions come from open data, which we reproject from o1280 (the original projection of IFS initial conditions) to n320 (AIFS resolution) by first converting them to a 0.25-degree grid. In the operational setup, however, data is reprojected directly from o1280 to n320. This difference in reprojection methods may lead to variations in the resulting input conditions, causing minor differences in the forecast.

# 4. Inspect the generated forecast

#### Plot a field

In [ ]:
# To be able to run the plotting section below you need to install additional dependencies

# !pip install -q matplotlib
# !pip install -q cartopy

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.tri as tri

In [ ]:
def fix(lons):
    # Shift the longitudes from 0-360 to -180-180
    return np.where(lons > 180, lons - 360, lons)

latitudes = state["latitudes"]
longitudes = state["longitudes"]
values = state["fields"]["100u"]

fig, ax = plt.subplots(figsize=(11, 6), subplot_kw={"projection": ccrs.PlateCarree()})
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=":")

triangulation = tri.Triangulation(fix(longitudes), latitudes)

contour=ax.tricontourf(triangulation, values, levels=20, transform=ccrs.PlateCarree(), cmap="RdBu")
cbar = fig.colorbar(contour, ax=ax, orientation="vertical", shrink=0.7, label="100u")

plt.title("100m winds (100u) at {}".format(state["date"]))
plt.show()

NameError: name 'state' is not defined